# 🧼 Data Cleaning & Wrangling Project
## 1. Project Overview
- **Dataset**: dirty_cafe_sales.csv
- **Source**: Kaggle Dataset - https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training/data
- **Goal**: This primary objective of this project was to identify and rectify these issues, producing a clean, reliable dataset suitable for further analysis and visualization. The process involved initial data exploration, identification of quality problems, development and implementation of a cleaning strategy, and final validation of the results.



In [3]:
# Import necessary libararies
import pandas as pd
import numpy as np

In [1]:
# Import Menu Item with Its Prices
menu_prices ={
    "Coffee": 2.0,
    "Tea": 1.5,
    "Sandwich": 4.0,
    "Salad": 5.0,
    "Cake": 3.0,
    "Cookie": 1.0,
    "Smoothie": 4.0,
    "Juice": 3.0
}

In [4]:
# Load Dataset
df = pd.read_csv('dirty_cafe_sales.csv')  # Change to your file
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


## 2. Initial Data Exploration

In [42]:
print("Rows:", df.shape[0], "Columns:", df.shape[1]) # we have 8 columns and 10000 rows of data in the dataset
df.info() # transaction ID is unique idetifier of each data entry

Rows: 10000 Columns: 8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 312.6+ KB


In [43]:
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [44]:
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

## 3. Data Cleaning Steps

### 3.1 Convert Data Types

In [45]:
# Transcation ID is the unique identifier, so no change is made for it.
# Item - remove whitespace
df['Item'] = df['Item'].str.strip() 

# Quantity, Price Per Unit, Total Spent should be converted to numeric
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')

# transaction date should be converted to date
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')

df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,NaN,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


### 3.2 Handling Invalid/Inconsistent Values

In [46]:
df.replace(['ERROR', 'UNKNOWN'], np.nan, inplace=True)

In [47]:
item_mode = df['Item'].mode()[0]
df['Item'] = df['Item'].fillna(item_mode)

payment_mode = df["Payment Method"].mode()[0]
df["Payment Method"] = df["Payment Method"].fillna(payment_mode)

location_mode = df["Location"].mode()[0]
df["Location"] = df["Location"].fillna(location_mode)

date_mode = df["Transaction Date"].mode()[0]
df["Transaction Date"] = df["Transaction Date"].fillna(date_mode)

quantity_median = df['Quantity'].median()
df['Quantity'] = df['Quantity'].fillna(quantity_median)

In [50]:
# fill in the empty values in price per unit based on the items
def impute_price(row):
    if pd.isna(row['Price Per Unit']):
        item = row['Item']
        if item in menu_prices:
            return menu_prices[item]
        else:
            price_median = df['Price Per Unit'].median()
            return price_median
    return row['Price Per Unit']

df['Price Per Unit'] = df.apply(impute_price, axis=1)

df['Total Spent'] = df['Quantity'] * df['Price Per Unit']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    10000 non-null  object        
 1   Item              10000 non-null  object        
 2   Quantity          10000 non-null  float64       
 3   Price Per Unit    10000 non-null  float64       
 4   Total Spent       10000 non-null  float64       
 5   Payment Method    10000 non-null  object        
 6   Location          10000 non-null  object        
 7   Transaction Date  10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 468.8+ KB


### 3.3 Remove Duplicates

In [51]:
initial_rows = len(df)
df.drop_duplicates(subset=['Transaction ID'], keep='first', inplace=True)
duplicates_removed = initial_rows - len(df)
if duplicates_removed > 0:
    print("Removed {duplicates_removed} duplicate rows based on 'Transaction ID'.")
else:
    print("No duplicate 'Transaction ID' found.")

No duplicate 'Transaction ID' found.


## 4. Post-Cleaning Checks

In [52]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

## 5. Summary & Next Steps
The data cleaning process successfully addressed the identified quality issues in the original dirty_cafe_sales.csv dataset. Missing values, incorrect data types, invalid entries, and calculation inconsistencies were handled according to the defined plan. Validation confirmed that the resulting dataset, cleaned_cafe_sales.csv, is complete, consistent, and has appropriate data types.
**This cleaned dataset is now suitable for reliable downstream analysis, visualization, or machine learning tasks**

In [53]:
# save cleaned data
df.to_csv("cleaned_cafe_, index=False)

<class 'NameError'>: name 'Cleaned_Cafe_sales' is not defined